In [1]:
import pandas as pd
import altair as alt
from statsmodels.nonparametric.smoothers_lowess import lowess
import numpy as np

In [2]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('favorability_polls.csv')

# Display the first few rows of the DataFrame to verify the import
df.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,imputed_sample_size,politician_id,politician,favorable,unfavorable,alternate_answers,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable
0,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,1248.0,42,Joe Biden,38.0,55.0,7.0,NaN,NaN,NaN,NaN
1,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,1248.0,11,Donald Trump,43.0,53.0,4.0,NaN,NaN,NaN,NaN
2,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,1248.0,13265,JD Vance,34.0,42.0,23.0,NaN,NaN,NaN,NaN
3,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,1248.0,6826,Kamala Harris,48.0,48.0,4.0,NaN,NaN,NaN,NaN
4,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,1248.0,3033,Tim Walz,36.0,35.0,29.0,NaN,NaN,NaN,NaN


In [3]:
df_state_na = df[df['state'].isna()]

df_state_not_na = df[df['state'].notna()]

print(df_state_na.head())
print(df_state_not_na.columns)
print(df_state_not_na.head())

   poll_id  pollster_id pollster sponsor_ids sponsors   display_name  \
0    88715          770     TIPP         NaN      NaN  TIPP Insights   
1    88715          770     TIPP         NaN      NaN  TIPP Insights   
2    88715          770     TIPP         NaN      NaN  TIPP Insights   
3    88715          770     TIPP         NaN      NaN  TIPP Insights   
4    88715          770     TIPP         NaN      NaN  TIPP Insights   

   pollster_rating_id pollster_rating_name  numeric_grade  pollscore  ...  \
0                 144        TIPP Insights            1.8       -0.4  ...   
1                 144        TIPP Insights            1.8       -0.4  ...   
2                 144        TIPP Insights            1.8       -0.4  ...   
3                 144        TIPP Insights            1.8       -0.4  ...   
4                 144        TIPP Insights            1.8       -0.4  ...   

  imputed_sample_size  politician_id     politician favorable unfavorable  \
0              1248.0      

In [4]:
df_state_na_clean = df_state_na[df_state_na['numeric_grade'].notna()]
print(df_state_na_clean['numeric_grade'])
print(df_state_na_clean['numeric_grade'].max())

0       1.8
1       1.8
2       1.8
3       1.8
4       1.8
       ... 
5543    3.0
5544    3.0
5545    3.0
5546    3.0
5547    1.9
Name: numeric_grade, Length: 5453, dtype: float64
3.0


In [5]:
# Make a copy to avoid the warning
df_state_na_clean = df_state_na_clean.copy()

# Now safely create the 'weight_score' column
df_state_na_clean.loc[:, 'weight_score'] = df_state_na_clean['numeric_grade'] / 3.0

In [6]:
print(df_state_na_clean['methodology'].unique())

['Online Panel' 'Online Panel/Text-to-Web' 'Probability Panel'
 'Live Phone' nan 'IVR/Online Panel/Text-to-Web'
 'Live Phone/Online Panel/Text-to-Web' 'Live Phone/Text-to-Web'
 'Live Phone/Online Panel/Text' 'Online Panel/Probability Panel'
 'Text-to-Web/Online Ad' 'IVR/Text' 'Online Ad' 'Live Phone/Online Panel'
 'Live Phone/Text/Online Panel' 'IVR/Online Panel/Email'
 'Live Phone/Probability Panel' 'IVR/Online Panel'
 'Online Panel/Online Ad' 'Text' 'IVR/Live Phone/Online Panel']


In [7]:
import numpy as np
# Make a copy to avoid the warning
df_state_na_clean = df_state_na_clean.copy()
# Mapping the weights to modes based on the table above
mode_weights = {
    'Live Phone': 1.00,
    'Live Phone/Probability Panel': 0.95,
    'Live Phone/Online Panel/Text-to-Web': 0.90,
    'Live Phone/Online Panel/Text': 0.90,
    'Live Phone/Text-to-Web/App Panel': 0.82,
    'Live Phone/Text-to-Web/Online Ad': 0.85,
    'Live Phone/Text-to-Web': 0.85,
    'Live Phone/Text/Online Panel': 0.90,
    'Live Phone/Online Panel': 0.85,
    'Live Phone/Online Panel/App Panel': 0.85,
    'IVR/Live Phone/Text/Online Panel/Email': 0.80,
    'Live Phone/Text/Online Ad': 0.80,
    'IVR/Online Panel/Email': 0.77,
    'IVR/Online Panel/Text-to-Web/Email': 0.75,
    'IVR/Online Panel/Text-to-Web': 0.75,
    'IVR/Online Panel': 0.70,
    'IVR': 0.70,
    'Online Panel/Probability Panel': 0.65,
    'Probability Panel': 0.65,
    'Online Panel/Text-to-Web': 0.60,
    'Online Panel/Online Ad': 0.55,
    'Online Panel': 0.50,
    'Online Ad': 0.50,
    'App Panel': 0.50,
    'Online Panel/Text-to-Web/Text': 0.50,
    'IVR/Text-to-Web': 0.50,
    'Text-to-Web/Online Ad': 0.45,
    'Text': 0.40,
    'IVR/Text': 0.40,
    'nan' : 0.50,
     np.nan: 0.50  # Handling missing or unknown values
}

# Apply the mapping to create a new column 'weight_mode'
df_state_na_clean.loc[:,'weight_mode'] = df_state_na_clean['methodology'].map(mode_weights)

In [8]:
print(df_state_na_clean.head())

   poll_id  pollster_id pollster sponsor_ids sponsors   display_name  \
0    88715          770     TIPP         NaN      NaN  TIPP Insights   
1    88715          770     TIPP         NaN      NaN  TIPP Insights   
2    88715          770     TIPP         NaN      NaN  TIPP Insights   
3    88715          770     TIPP         NaN      NaN  TIPP Insights   
4    88715          770     TIPP         NaN      NaN  TIPP Insights   

   pollster_rating_id pollster_rating_name  numeric_grade  pollscore  ...  \
0                 144        TIPP Insights            1.8       -0.4  ...   
1                 144        TIPP Insights            1.8       -0.4  ...   
2                 144        TIPP Insights            1.8       -0.4  ...   
3                 144        TIPP Insights            1.8       -0.4  ...   
4                 144        TIPP Insights            1.8       -0.4  ...   

      politician  favorable  unfavorable alternate_answers very_favorable  \
0      Joe Biden       38.0

In [9]:
# Count the number of NaN values in the 'sample_size' column
nan_count = df['sample_size'].isna().sum()

print(f"Number of NaN values in 'sample_size': {nan_count}")

# Calculate the mean of the available (non-NaN) sample sizes
mean_sample_size = df['sample_size'].mean()

print(f"Mean of available sample sizes: {mean_sample_size}")

Number of NaN values in 'sample_size': 14
Mean of available sample sizes: 1646.3202023852548


In [10]:
# Step 2: Create the 'weight_sample' column
df_state_na_clean['weight_sample'] = df_state_na_clean['sample_size'].apply(lambda x: np.sqrt(x) if not np.isnan(x) else np.sqrt(mean_sample_size))

# Display the first few rows to verify
df_state_na_clean.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,favorable,unfavorable,alternate_answers,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,weight_score,weight_mode,weight_sample
0,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,38.0,55.0,7.0,NaN,NaN,NaN,NaN,0.6,0.5,35.327043
1,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,43.0,53.0,4.0,NaN,NaN,NaN,NaN,0.6,0.5,35.327043
2,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,34.0,42.0,23.0,NaN,NaN,NaN,NaN,0.6,0.5,35.327043
3,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,48.0,48.0,4.0,NaN,NaN,NaN,NaN,0.6,0.5,35.327043
4,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,36.0,35.0,29.0,NaN,NaN,NaN,NaN,0.6,0.5,35.327043


In [11]:
# Convert 'end_date' to datetime format with specified format for single/double digits in month/day
df_state_na_clean['end_date'] = pd.to_datetime(df_state_na_clean['end_date'], format='%m/%d/%y', errors='coerce')

# Sort the DataFrame by 'end_date'
df_state_na_clean_sorted = df_state_na_clean.sort_values(by='end_date',ascending=False)

In [12]:
df_state_na_clean_sorted.head()

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,numeric_grade,pollscore,...,favorable,unfavorable,alternate_answers,very_favorable,somewhat_favorable,somewhat_unfavorable,very_unfavorable,weight_score,weight_mode,weight_sample
0,88715,770,TIPP,NaN,NaN,TIPP Insights,144,TIPP Insights,1.8,-0.4,...,38.0,55.0,7.0,NaN,NaN,NaN,NaN,0.6,0.5,35.327043
11,88710,568,YouGov,352,Economist,YouGov,391,YouGov,3.0,-1.1,...,41.0,56.0,3.0,19.0,22.0,15.0,41.0,1.0,0.5,40.298883
19,88710,568,YouGov,352,Economist,YouGov,391,YouGov,3.0,-1.1,...,49.0,46.0,5.0,30.0,19.0,10.0,37.0,1.0,0.5,36.290495
18,88710,568,YouGov,352,Economist,YouGov,391,YouGov,3.0,-1.1,...,48.0,47.0,5.0,29.0,19.0,10.0,36.0,1.0,0.5,38.170669
17,88710,568,YouGov,352,Economist,YouGov,391,YouGov,3.0,-1.1,...,43.0,44.0,13.0,25.0,18.0,12.0,32.0,1.0,0.5,40.298883


In [13]:
# Step 3: Get the first (top) date after sorting
first_date = df_state_na_clean_sorted['end_date'].iloc[0]

# Step 4: Compute the difference in days and create the 'days_past_index' column
df_state_na_clean_sorted['days_past_index'] = (first_date - df_state_na_clean_sorted['end_date']).dt.days

In [14]:
# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_na_clean_sorted_cutoff = df_state_na_clean_sorted[df_state_na_clean_sorted['end_date'] >= cutoff_date].copy()

# Check for NaN values in 'weight_mode', 'weight_sample', and 'weight_score'
nan_check = df_state_na_clean_sorted_cutoff[['weight_mode', 'weight_sample', 'weight_score']].isna().sum()

# Step 2: Check for infinite values in the involved columns
inf_check = df_state_na_clean_sorted_cutoff[['weight_mode', 'weight_sample', 'weight_score']].isin([np.inf, -np.inf]).sum()
print(f"Number of infinite values:\n{inf_check}")


# Print the result to verify if there are any NaN values
print(nan_check)

# Step 1: Filter rows where 'weight_mode' is NaN
nan_weight_mode = df_state_na_clean_sorted_cutoff[df_state_na_clean_sorted_cutoff['weight_mode'].isna()]

# Step 2: Display the 'methodology' or other relevant columns to investigate the methodology used
# For example, we'll check 'pollster', 'sponsors', and 'methodology' (if available) along with 'weight_mode'
nan_weight_mode_info = nan_weight_mode[['pollster', 'sponsors', 'methodology', 'weight_mode']]

# Print the resulting DataFrame for verification
print(nan_weight_mode_info)

Number of infinite values:
weight_mode      0
weight_sample    0
weight_score     0
dtype: int64
weight_mode      0
weight_sample    0
weight_score     0
dtype: int64
Empty DataFrame
Columns: [pollster, sponsors, methodology, weight_mode]
Index: []


In [15]:
# Initialize lambda and create an empty DataFrame to store the results
lambda_value = 1.0
weighted_averages = []

# Define the two candidate IDs of interest
politician_ids = [6826, 11]

# Define the cut-off date for polls (July 25, 2024)
cutoff_date = pd.to_datetime('2024-07-25')

# Step 2: Filter to exclude polls with an 'end_date' before July 25, 2024
df_state_na_clean_sorted_cutoff = df_state_na_clean_sorted[df_state_na_clean_sorted['end_date'] >= cutoff_date].copy()
df_lv = df_state_na_clean_sorted_cutoff[df_state_na_clean_sorted_cutoff['population']=='lv'].copy()

# Iterate through each unique end date
for current_date in df_lv['end_date'].unique():
    
    # Step 2: Filter the data for polls on or before the current end date
    current_data = df_lv[df_lv['end_date'] <= current_date].copy()
    
    # Step 3: Compute 'days_past_index' as the difference between current_date and each poll's end_date
    current_data['days_past_index'] = (current_date - current_data['end_date']).dt.days
    
    # Step 4: Compute 'weight_time' using the formula exp(-lambda * t)
    current_data['weight_time'] = np.exp(-lambda_value * current_data['days_past_index'])

    # Check for NaN entries in the 'weight_time' column
    #nan_weight_time = current_data['weight_time'].isna().sum()
    
    # Print the result
    #print(f"Number of NaN values in 'weight_time': {nan_weight_time}")

    # Step 2: Check for infinite values in the involved columns
    #inf_check = current_data[['weight_time']].isin([np.inf, -np.inf]).sum()
    #print(f"Number of infinite values:\n{inf_check}")
    
    # Step 6: Iterate through each unique candidate to compute the weighted average for that candidate
    for politician in politician_ids:
        #  We select H2H if avaible and only use non H2H if H2H is not available
        candidate_data = current_data[current_data['politician_id'] == politician].copy()
        
        # Step 7: Filter data for the specific candidate
        
        c_mean = candidate_data[candidate_data['days_past_index']< 30]['favorable'].mean() #gather mean for past 30 days of likely voters
        c_std =  candidate_data[candidate_data['days_past_index']< 30]['favorable'].std()  #gather standard deviation for past 30 days of likely voters
        candidate_data['zscores'] = abs((candidate_data['favorable']-c_mean)/c_std)
        candidate_data['weight_outlier'] = np.exp(-1.0*candidate_data['zscores'])              
        # Step 8: Compute the total weight 'w_i' for each poll
        #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
        candidate_data.loc[:, 'w_i'] = (
            candidate_data['weight_mode'] * candidate_data['weight_sample'] *
            candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']
        )
        #Start House effect data computation#
        c_mean_by_pollster = candidate_data[(candidate_data['days_past_index'] < 15) & 
                                            (candidate_data['days_past_index'] >= 0)].groupby('pollster_id')['favorable'].mean()
        c_mean_by_pollster_u = candidate_data[candidate_data['days_past_index'] < 15].groupby('pollster_id')['unfavorable'].mean()
        
        c_mean2 = candidate_data[candidate_data['days_past_index'] < 15]['favorable'].mean()
        c_mean2_u = candidate_data[candidate_data['days_past_index'] < 15]['unfavorable'].mean()
        # Convert to a DataFrame for easier manipulation
        c_mean_by_pollster_df = c_mean_by_pollster.reset_index()
        c_mean_by_pollster_df_u = c_mean_by_pollster_u.reset_index()
        # Check for NaN values in the entire DataFrame
        #nan_check = c_mean_by_pollster_df.isna().sum()
        #print(c_mean)
        
        # Display the result
        #print(nan_check)
        # Calculate the house effect for each pollster (difference from overall mean)
        c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['favorable'] - c_mean2
        c_mean_by_pollster_df_u['house_effectu'] = c_mean_by_pollster_df_u['unfavorable'] - c_mean2_u
        # Fill NaN values with 0 in the 'house_effect' column
        #c_mean_by_pollster_df['house_effect'].fillna(0, inplace=True)
        
        c_mean_by_pollster_df['house_effect'] = c_mean_by_pollster_df['house_effect'].fillna(0)
        c_mean_by_pollster_df_u['house_effectu'] = c_mean_by_pollster_df_u['house_effectu'].fillna(0)
        #print(c_mean_by_pollster_df.head())
        # Merge the house effect back into the original data
        candidate_data = candidate_data.merge(c_mean_by_pollster_df[['pollster_id', 'house_effect']], on='pollster_id', how='left')
        candidate_data = candidate_data.merge(c_mean_by_pollster_df_u[['pollster_id', 'house_effectu']], on='pollster_id', how='left')
        candidate_data['house_effect']=candidate_data['house_effect'].fillna(0)
        candidate_data['house_effectu']=candidate_data['house_effectu'].fillna(0)
        #print(candidate_data.head(10))
        # Apply the house effect to adjust the 'pct' values
        candidate_data['adjusted_favorable'] = candidate_data['favorable'] - candidate_data['house_effect']
        candidate_data['adjusted_unfavorable'] = candidate_data['unfavorable'] - candidate_data['house_effectu']
        #print(candidate_data['adjusted_pct'].head(10))
        # Step 8: Compute the total weight 'w_i' for each poll
        #candidate_data['weight_mode'] = 1.0  # Assuming 'weight_mode' is a fixed value of 1.0, adjust if needed
        candidate_data.loc[:, 'w_i'] = (
            candidate_data['weight_mode'] * candidate_data['weight_sample'] *
            candidate_data['weight_score'] * candidate_data['weight_time'] * candidate_data['weight_outlier']
        )
        # check for NaN entries in the w_i column
        #nan_weight_i = candidate_data['w_i'].isna().sum()
        #print result
        #print(f"Number of NaN value in 'w_i': {nan_weight_i}")
        
        # Step 3: Identify rows with NaN in w_i to examine their individual values
        #nan_rows = candidate_data[candidate_data['w_i'].isna()]
        #print("Rows producing NaN in 'w_i':")
        #print(nan_rows[['methodology','weight_mode', 'weight_sample', 'weight_score', 'weight_time', 'w_i']])
        
        # Step 9: Compute the weighted average of the poll points ('pct') using the computed weights
        weighted_average_favorable = np.average(candidate_data['adjusted_favorable'], weights=candidate_data['w_i'])
        weighted_average_unfavorable = np.average(candidate_data['adjusted_unfavorable'], weights=candidate_data['w_i'])
        # Step 10: Store the current date, candidate_id, candidate_name, and the weighted average in a new DataFrame
        if not candidate_data.empty:
            candidate_name = candidate_data['politician'].iloc[0]
            weighted_averages.append({
                'end_date': current_date,
                'politician_id': politician,
                'candidate_name': candidate_name,
                'weighted_average_favorable': weighted_average_favorable,
                'weighted_average_unfavorable':weighted_average_unfavorable
            })

# Convert the results into a new DataFrame
df_weighted_averages = pd.DataFrame(weighted_averages)    
df_weighted_averages.head()



,end_date,politician_id,candidate_name,weighted_average_favorable,weighted_average_unfavorable
0,2024-10-15,6826,Kamala Harris,51.087776,47.843113
1,2024-10-15,11,Donald Trump,44.604483,53.785352
2,2024-10-13,6826,Kamala Harris,50.710118,47.299983
3,2024-10-13,11,Donald Trump,46.220339,52.802738
4,2024-10-10,6826,Kamala Harris,49.933027,47.993410


In [16]:
# Create a customized color encoding for favorable and unfavorable
color_scale = alt.Scale(domain=['Donald Trump - Favorable', 'Donald Trump - Unfavorable', 
                                'Kamala Harris - Favorable', 'Kamala Harris - Unfavorable'], 
                        range=['green', 'red', 'blue', 'orange'])

# Transform the data into a longer format to handle both favorable and unfavorable
df_long = df_weighted_averages.melt(id_vars=['end_date', 'candidate_name'],
                                    value_vars=['weighted_average_favorable', 'weighted_average_unfavorable'],
                                    var_name='sentiment', value_name='percentage')
#print(df_long.head(100))

# Create a column to differentiate between favorable and unfavorable for each candidate
df_long['candidate_sentiment'] = df_long['candidate_name'] + ' - ' + df_long['sentiment'].apply(lambda x: 'Unfavorable' if 'unfavorable' in x else 'Favorable')

print(df_long.head(100))

# Scatter plot for individual data points
scatter = alt.Chart(df_long).mark_point(size=50, opacity=0.7).encode(
    x='end_date:T',  # Temporal encoding for end_date
    y='percentage:Q',  # Quantitative encoding for favorable/unfavorable percentage
    color=alt.Color('candidate_sentiment:N', scale=color_scale),  # Custom color scale for candidates and sentiment
    tooltip=['end_date', 'candidate_name', 'percentage', 'sentiment']  # Add tooltips to show details
)

# LOESS curves for trend lines
loess = alt.Chart(df_long).transform_loess('end_date', 'percentage', groupby=['candidate_sentiment']).mark_line(size=3).encode(
    x='end_date:T',
    y='percentage:Q',
    color=alt.Color('candidate_sentiment:N', scale=color_scale)  # Use same color scale for the curves
)

# Combine scatter plot and LOESS curves
chart = scatter + loess

# Set chart properties
chart = chart.properties(
    title='Favorable vs Unfavorable Polling Results Over Time with LOESS Curves',
    width=600,
    height=400
)

chart.show()


     end_date candidate_name                     sentiment  percentage  \
0  2024-10-15  Kamala Harris    weighted_average_favorable   51.087776   
1  2024-10-15   Donald Trump    weighted_average_favorable   44.604483   
2  2024-10-13  Kamala Harris    weighted_average_favorable   50.710118   
3  2024-10-13   Donald Trump    weighted_average_favorable   46.220339   
4  2024-10-10  Kamala Harris    weighted_average_favorable   49.933027   
..        ...            ...                           ...         ...   
95 2024-10-01   Donald Trump  weighted_average_unfavorable   52.650396   
96 2024-09-29  Kamala Harris  weighted_average_unfavorable   47.693843   
97 2024-09-29   Donald Trump  weighted_average_unfavorable   53.300573   
98 2024-09-25  Kamala Harris  weighted_average_unfavorable   47.757387   
99 2024-09-25   Donald Trump  weighted_average_unfavorable   52.890317   

            candidate_sentiment  
0     Kamala Harris - Favorable  
1      Donald Trump - Favorable  
2     Kam

alt.LayerChart(...)